In [1]:
import illustris_python as il
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import SymLogNorm
import h5py
from tqdm import tqdm


In [2]:
basePath = 'sims.TNG/TNG100-1/output'
snap     = 99
halo_ID  = 172650

In [1]:
!ls

examples	  moreColors.png		  sims.other
illustris_idl	  progenitorID_subhalo172650.txt  sims.TNG
illustris_julia   progenitorID_subhalo239993.txt  SubhaloID_stars.txt
illustris_matlab  README.txt
illustris_python  sims.illustris


In [4]:
fields = ['Coordinates','ParticleIDs']
stars  = il.snapshot.loadSubhalo(basePath,snap, halo_ID, 'stars', fields=fields)
tree   = il.sublink.loadTree(basePath, snap, halo_ID, fields=['SubfindID','SnapNum'])

starsID = stars['ParticleIDs']

numsnap = np.arange(0,99 + 1,1)

progenitorID = np.zeros(stars['count']) - 1

subhalosID = tree['SubfindID']
subhalosSnap  = tree['SnapNum'] 
# # Iterate on the snapshots
for s in tqdm(numsnap, leave=True):

    # Iterate on the subhalos in that snapshot ONLY
    idx_subhalos  = np.where(subhalosSnap == s)[0]
    keep_subhalos = subhalosID[idx_subhalos]

    for i in keep_subhalos:

        # Load ID of stars in the subhalo
        subhalos_starsID = il.snapshot.loadSubhalo(basePath,s, i, 'stars', fields=['ParticleIDs'])

        not_already_IDed = np.where(progenitorID == -1)[0]
        stars_in = np.isin(starsID[not_already_IDed], subhalos_starsID)

        indices = not_already_IDed[stars_in]
        progenitorID[indices] = i
        
np.savetxt(f'progenitorID_subhalo{halo_ID}.txt', progenitorID, fmt='%d')

100%|██████████| 100/100 [55:35<00:00, 33.36s/it] 
